In [ ]:
## Import necessay libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import IsolationForest
import pandas as pd
# define the directory containing the files
dir_path = 'C:/Users/Pratibha/Downloads/happy/Dataset-20230227T042340Z-001/Dataset/s3Files/'

# get a list of all the files in the directory
file_list = os.listdir(dir_path)

# loop over each file
for file_name in file_list:
    # check if the file is a text file
    if file_name.endswith('.txt'):
        # read the contents of the file
        with open(os.path.join(dir_path, file_name), 'r') as f:
            file_contents = f.read()


In [ ]:
file_list
# file_contents

In [ ]:
# example unstructured numerical data
# unstructured_data = '1,2,3\n4,5,6\n7,8,9'

# split the unstructured data by newline characters to separate rows
rows = file_contents.split('\n')

# loop over each row, split it by comma characters to separate values, and convert the values to integers
structured_data = []
for row in rows:
    values = row.split(',')
    structured_row = [str(value) for value in values]
    structured_data.append(structured_row)

# print the structured data
print(structured_data)


In [ ]:
df=pd.DataFrame(structured_data,columns=[0,'p1','p2','p3',4]).drop(columns=[0,4])
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
# !pip3 install matplotlib
# !pip install pillow
# !pip install --upgrade pip

In [ ]:
df.info()

In [ ]:
df=df.astype('float')
df.info()

In [ ]:
df.head()

In [ ]:
# import random

# # Define the number of columns
# num_columns = 1

# # Define the number of rows
# num_rows = 10000

# # Create an empty list to hold the columns
# columns = [[] for _ in range(num_columns)]

# # Generate random times for each column
# for col in range(num_columns):
#     for row in range(num_rows):
#         # Generate a random time between 0 and 59 seconds
#         time_in_seconds = random.randint(0, 59)
#         columns[col].append(time_in_seconds)



In [ ]:
import random

# Define the number of columns
num_columns = 1

# Define the number of rows
num_rows = 10000

# Create an empty list to hold the columns
columns = [[] for _ in range(num_columns)]

# Generate random times for each column
for col in range(num_columns):
    for row in range(num_rows):
        # Generate a random time between 0 and 60 seconds
        seconds = random.randint(0, 59)

        # Generate a random number of minutes between 0 and 59
        minutes = random.randint(0, 59)

        # Generate a random number of hours between 0 and 1
        hours = random.uniform(0, 1)

        # Convert the time to a decimal number of hours and add it to the column
        time_in_hours = hours + (minutes / 60) + (seconds / 3600)
        columns[col].append(time_in_hours)


In [ ]:
df['time_insec']=round(pd.DataFrame(columns).T, 2)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# import random

# # Generate 1000 rows of time data in seconds
# time_data = [random.randint(0, 3600) for _ in range(1000)]
# time_data

In [ ]:
# Create an anomaly by setting a single value to a very large number (e.g. 99999)
anomaly_index = 500
d=df['time_insec']
d[anomaly_index] = 99999


In [ ]:
# Define a threshold for anomaly detection (e.g. 1 hour)
threshold = 60

# Test whether any value in the data is greater than the threshold
if any(time > threshold for time in d):
    print("Anomaly detected!")
else:
    print("No anomalies detected.")


In [ ]:
data = pd.read_csv('frequencyFeatures.csv')
data.head(2)

In [ ]:
data.columns

In [ ]:
data.groupby('Label')['Timestamp', 'xAcc010Hz', 'xAcc015Hz', 'xAcc020Hz',
       'xAcc025Hz', 'xAcc030Hz', 'xAcc035Hz', 'xAcc040Hz','snd2275Hz', 'snd2300Hz', 'snd2325Hz', 'snd2350Hz', 'snd2375Hz',
       'snd2400Hz', 'snd2425Hz', 'snd2450Hz', 'snd2475Hz', 'snd2500Hz'].median()

In [ ]:
data.shape
df['time_insec'].mean()

In [ ]:
df.median()

In [ ]:
df[df['time_insec']==df['time_insec'].median()].mean()

In [ ]:

# import graph objects as "go"
import plotly.graph_objs as go
import plotly.express as px

fig = px.scatter(df,x = 'p2',y = 'p3',
             title = 'phases',
            template = "plotly_dark")

fig.show()

In [ ]:
fig = px.scatter(df,x = 'p1',y = 'p3',
             title = 'phases',
            template = "plotly_dark")

fig.show()

In [ ]:
fig = px.scatter(df,x = 'p1',y = 'p2',
             title = 'phases',
            template = "plotly_dark")

fig.show()

In [ ]:
print("Missing Data is", data.isnull().sum().sum())
data.info()

In [ ]:
y= data.iloc[:,1]
data=data.iloc[:,2:]
y.unique()

In [ ]:
data.head()

In [ ]:
data.info()


In [ ]:
model= IsolationForest(n_estimators=100, max_samples='auto', contamination= float(0.2), max_features=1.0)

In [ ]:
model.fit(df.iloc[:,0:3])

In [ ]:
df['anomaliesscore']= model.decision_function(df.iloc[:,0:3])
df['anomaly']= model.predict(df.iloc[:,0:3])
df

In [ ]:
df[['anomaliesscore','anomaly']].describe().T

In [ ]:

from sklearn.cluster import KMeans

# load the data
# data = pd.read_csv('machine_data.csv')

# preprocess the data
X = df.drop(columns=['time_insec','anomaliesscore','anomaly'], axis=1) # drop the machine id column
X_norm = (X - X.mean()) / X.std() # normalize the data


In [ ]:
from sklearn.cluster import KMeans
wcss = []

for i in range(1, 21):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', 
                    max_iter = 300, n_init = 10, random_state = 0).fit(X_norm)
    wcss.append(kmeans.inertia_)

# Plotting the results onto a line graph, 
# `allowing us to observe 'The elbow'
plt.plot(range(1, 21), wcss,marker='o',color='b')
plt.title('The elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS') # Within cluster sum of squares
plt.grid()
plt.show()


#### Number of clusters=6

In [ ]:
# Applying kmeans to the dataset / Creating the kmeans classifier
kmeans = KMeans(n_clusters = 6, init = 'k-means++',
                max_iter = 300, n_init = 10, random_state = 0)
y_kmeans = kmeans.fit_predict(X)

In [ ]:
# identify potential machine failures
cluster_centers = kmeans.cluster_centers_
distances = np.linalg.norm(X_norm - cluster_centers[kmeans.labels_], axis=1) # calculate distances to cluster centers
df['distance'] = distances # add distance column to data

In [ ]:
round(df['distance'].median())

In [ ]:
potential_failures = df[df['distance'] > round(df['distance'].median())] # identify machines with distance > 2

In [ ]:
potential_failures

In [ ]:
sns.set_style("whitegrid") 
sns.FacetGrid(df,hue='anomaly',height = 6).map(plt.scatter,'p1','p2')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:,1], 
            s = 100, c = 'yellow', label = 'Centroids')

plt.legend()


In [ ]:
sns.set_style("whitegrid") 
sns.FacetGrid(df,hue='anomaly',height = 6).map(plt.scatter,'p2','p3')
plt.scatter(kmeans.cluster_centers_[:, 1], kmeans.cluster_centers_[:,-1], 
            s = 100, c = 'yellow', label = 'Centroids')

plt.legend()

In [ ]:
sns.set_style("whitegrid") 
sns.FacetGrid(df,hue='anomaly',height = 6).map(plt.scatter,'p1','p3')
plt.scatter(kmeans.cluster_centers_[:, 1], kmeans.cluster_centers_[:,2], 
            s = 100, c = 'yellow', label = 'Centroids')

plt.legend()

In [ ]:
pd.DataFrame(y_kmeans).value_counts()

In [ ]:
df['anomaly'].value_counts()

In [ ]:
y_kmeans
relabel = np.choose(kmeans.labels_, [1, 1, 1,1,-1, -1]).astype(np.int64)
relabel
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(df['anomaly'],relabel))

In [ ]:
cluster_data = []
for i in range(kmeans.n_clusters):
    cluster_data.append(X[kmeans.labels_ == i])

In [ ]:
cluster_data

In [ ]:
kmeans

In [ ]:
df['cluster'] = y_kmeans
df.head()

In [ ]:
len(df)*0.5

In [ ]:
from scipy import signal
# # Filter data
# order = 3
# low_pass_freq = 1000
# sampling_freq = 10000
# b, a = signal.butter(order, low_pass_freq/(sampling_freq/2), 'lowpass')
# padlen = 2 * len(df.iloc[:500,0:3]) - 1
# filtered_data = signal.filtfilt(b, a, df.iloc[:500,0:3], padlen=padlen)

# # Compute FFT
# n = len(filtered_data)
# fft_data = abs(np.fft.fft(filtered_data))/n
# freq = np.fft.fftfreq(n, d=1/sampling_freq)
# freq = freq[:n//2]

# # Plot FFT
# plt.plot(freq, fft_data[:, 0][:n//2])
# plt.xlabel('Frequency (Hz)')
# plt.ylabel('Amplitude')
# plt.show()


In [ ]:
# Perform MCSA analysis
window_size = 1024
fft_size = 1024
overlap = 0.5

current_rms = []
for i in range(0, len(df.iloc[:1000,0:2]), window_size):
    window = data[i:i+window_size]
    fft = np.fft.fft(window, fft_size)
    freqs = np.fft.fftfreq(fft_size, 1 / len(window))
    rms = np.sqrt(np.mean(np.abs(fft)**2, axis=1))
    current_rms.append(rms)

current_rms = np.array(current_rms)

# Plot MCSA results
plt.figure(figsize=(10, 5))
plt.imshow(current_rms.T, aspect='auto', cmap='jet', origin='lower')
plt.xlabel('Window index')
plt.ylabel('Frequency (Hz)')
plt.colorbar()
plt.title('Motor current signature analysis')
plt.show()

# # Load vibration data
# vibration_data = pd.read_csv('vibration_data.csv')

# Perform model-based VI analysis
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
vibration_pca = pca.fit_transform(df)

# Plot model-based VI results
plt.figure(figsize=(10, 5))
plt.scatter(vibration_pca[:, 0], vibration_pca[:, 1])
plt.xlabel('Principal component 1')
plt.ylabel('Principal component 2')
plt.title('Model-based vibration analysis')
plt.show()

# Predict machine failure
threshold_current = 0.5
threshold_vibration = 2.0

faults_current = np.where(current_rms > threshold_current)
faults_vibration = np.where(vibration_pca > threshold_vibration)

if len(faults_current[0]) > 0 or len(faults_vibration[0]) > 0:
    print('Machine failure predicted.')
else:
    print('Machine failure not predicted.')


In [ ]:
from scipy.signal import butter, lfilter

# Set filter parameters
order = 2
low_pass_freq = 100
sampling_freq = 1000

# Compute filter coefficients
nyquist_freq = 0.5 * sampling_freq
cutoff_freq = low_pass_freq / nyquist_freq
b, a = butter(order, cutoff_freq, btype='lowpass', analog=False)

# Filter data
filtered_data = lfilter(b, a, df[['p1', 'p2', 'p3']], axis=0)

# Compute FFT
n = len(filtered_data)
fft_data = abs(np.fft.fft(filtered_data)) / n
freq = np.fft.fftfreq(n, d=1 / sampling_freq)
freq = freq[:n // 2]

# Plot FFT
plt.plot(freq, fft_data[:, 0][:n // 2])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.show()


In [ ]:
current_data = pd.read_csv('SynchronousMachine.csv')
current_data.head(2)

In [ ]:
current_data.shape

- Attribute Information

    1) I_y: Load Current
    
    2) PF: Power factor
    
    3) e_PF: Power factor error
    
    4) d_if: Changing of excitation current of synchronous machine
    
    5) I_f: Excitation current of synchronous machine

In [ ]:
# Perform MCSA analysis
window_size = 1024
fft_size = 1024
overlap = 0.5
current_data=df.iloc[:1000,0:3]
current_rms = []
for i in range(0, len(current_data), window_size):
    window = current_data[i:i+window_size]
    fft = np.fft.fft(window, fft_size)
    freqs = np.fft.fftfreq(fft_size, 1 / len(window))
    rms = np.sqrt(np.mean(np.abs(fft)**2, axis=1))
    current_rms.append(rms)

current_rms = np.array(current_rms)

# Plot MCSA results
plt.figure(figsize=(10, 5))
plt.imshow(current_rms.T, aspect='auto', cmap='jet', origin='lower')
plt.xlabel('Window index')
plt.ylabel('Frequency (Hz)')
plt.colorbar()
plt.title('Motor current signature analysis')
plt.show()

# # Load vibration data
# vibration_data = pd.read_csv('vibration_data.csv')

# Perform model-based VI analysis
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
vibration_pca = pca.fit_transform(current_data)

# Plot model-based VI results
plt.figure(figsize=(10, 5))
plt.scatter(vibration_pca[:, 0], vibration_pca[:, 1])
plt.xlabel('Principal component 1')
plt.ylabel('Principal component 2')
plt.title('Model-based vibration analysis')
plt.show()

# Predict machine failure
threshold_current = 0.5
threshold_vibration = 2.0

faults_current = np.where(current_rms > threshold_current)
faults_vibration = np.where(vibration_pca > threshold_vibration)

if len(faults_current[0]) > 0 or len(faults_vibration[0]) > 0:
    print('Machine failure predicted.')
else:
    print('Machine failure not predicted.')
